In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

data_dir = Path('./data')
data_dir.mkdir(parents=True, exist_ok=True)

rssi_threshold = 40

df_signal = pd.read_csv(Path('./data/signal_data.csv'), header=None)
# rssi(%)를 string에서 int로 변환(%)
df_signal[1] = pd.to_numeric(df_signal[1])
df_signal = df_signal[df_signal[1] > rssi_threshold]

bssid_set = np.unique(df_signal[0])
rp_set = np.unique(df_signal[4])
timestamp_set = np.unique(df_signal[2])

print(rp_set)

['2층 입구' 'LLOYD' 'ZARA' 'ZIOZIA' '버스정류장' '서점']


In [2]:
rp_encoder = LabelEncoder()
bssid_encoder = LabelEncoder()

# string으로 되어있는 rp와 bssid들을 숫자로 mapping
rp_encoder.fit(rp_set)
#rp_set_encoded = rp_encoder.transform(rp_set)
#rp_set_decoded = rp_encoder.inverse_transform(rp_set_encoded)
np.save('./data/classes.npy', rp_encoder.classes_)

bssid_encoder.fit(bssid_set)
np.save('./data/features.npy', bssid_encoder.classes_)


In [3]:
train_data = pd.DataFrame()

for rp in np.unique(df_signal[4]):
    # 각 rp에 해당하는 signal 추출
    rp_data = df_signal[df_signal[4] == rp]
    rp_num = rp_encoder.transform([rp_data.iloc[0][4]])[0]

    # key : timestamp, value : bssid-rssi, rp
    scan_dict = defaultdict(lambda : defaultdict(int).fromkeys(bssid_set, 0))
    for idx, signal in rp_data.iterrows():
        #         timestamp  bssid        rssi
        scan_dict[signal[2]][signal[0]] = signal[1]

    for timestamp, scan_data in scan_dict.items():
        scan_data['rp'] = rp_num
    
    train_data = train_data.append(pd.DataFrame.from_dict(scan_dict).transpose())


cols = train_data.columns.tolist()
cols.sort()
train_data = train_data[cols]

train_data.to_csv(Path('./data/train_data.csv'), index=False)
train_data

,00:07:79:0a:15:be,00:0f:00:2b:4f:6c,00:1d:93:89:d5:13,00:e0:4b:d7:04:c4,00:e1:6d:a4:62:e0,00:e1:6d:a4:ba:40,04:0e:3c:fd:f6:d5,04:5e:a4:d4:99:13,04:92:26:6b:13:a0,04:92:26:6b:13:a4,...,fc:0a:81:9a:64:41,fc:0a:81:9a:96:90,fc:0a:81:9a:96:91,fc:0a:81:9a:98:a0,fc:0a:81:9a:98:a1,fc:0a:81:9a:98:a2,fe:cb:ac:34:26:8f,fe:cb:bc:34:26:8f,fe:cb:bc:34:27:5a,rp
2020-11-26 21:30:22.159452,0,0,0,0,0,0,60,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-11-26 21:30:26.219957,0,0,0,0,0,0,53,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-11-26 21:30:30.297193,0,0,0,0,0,0,72,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-11-26 21:30:34.352415,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-11-26 21:30:38.379748,65,0,0,0,0,0,0,0,0,0,...,0,0,0,0,50,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-26 21:46:21.491170,62,0,0,0,0,0,0,0,0,0,...,0,46,0,67,0,0,0,0,0,5
2020-11-26 21:46:25.403357,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2020-11-26 21:46:29.325100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,67,0,75,0,0,0,5
2020-11-26 21:46:33.261422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,75,65,0,0,0,0,5
